In [14]:


import os
import pickle
from tqdm.notebook import tqdm
# Set the brt-devkit environment to `prod`
os.environ['BRT_ENV'] = 'prod'
# Set the directory where assets will be downloaded. brtdevkit will automatically use this directory
# to download dataset assets if a path is not passed. useful for when you have assets downloaded already
# so that assets are not downloaded again if already there
os.environ['ASSET_DIR'] = '~/.brt'  # this is the value by default
import brtdevkit
# from brtdevkit.data import Dataset
# from brtdevkit.core.db.athena import AthenaClient, Table
# import sys
# import boto3, imageio, io
sys.path.append('/home/aravindvenugopal/JupiterCVML/europa/base/src/europa')
brtdevkit.log_level = 'debug'
import numpy as np
import pandas as pd
from datetime import datetime
from helpers.transforms import compute_transforms_all
import time
import matplotlib.pyplot as plt
from dl.utils.io_utils import normalize_image
from helpers.spatiotemp_helpers import get_dataframe, image_from_s3, get_groups, store_data_single_frame, store_data_sequence
from helpers.spatiotemp_viz import visualize_data#image, visualize_cloud, fuse_cloud, visualize_fused_cloud
s3 = boto3.resource('s3')
%matplotlib inline
# from mpl_toolkits.mplot3d import Axes3D
from aletheia_dataset_creator.dataset_tools.aletheia_dataset_helpers import *
from aletheia_dataset_creator.config.dataset_config import * 
%load_ext autoreload
%autoreload 2
time_series = True



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
dataset_idx = 0
directory = [f'{os.environ["DATASET_PATH"]}/halo_all_cam_labeled_dataset', '/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq', '/data/jupiter/datasets/iq_2023_v5_anno'][dataset_idx]
csv_name = ['master_annotations.csv', '653a7a0a3c2d8ab221f6d915_master_annotations.csv','64dfcc1de5a41169c7deb205_master_annotations.csv'][dataset_idx]

save_path = directory + "/sp_samples_" #to save output of PP
pp_path = directory + "/processed/images/"  #dir containing PP outputs

In [16]:
halo_mcsv_path = directory + '/' + csv_name
halo_mcsv = pd.read_csv(halo_mcsv_path).drop_duplicates()

In [12]:
loc_dict =  {'T01': 0, 'T02': 1, 'T03': 2,'T04': 4,
'T05': 5, 'T06': 6, 'T07': 7, 'T08': 8,
'T09': 9, 'T10': 10, 'T11': 11,'T12': 12,
'T13': 13, 'T14': 14, 'T15': 15, 'T16': 16}

keys = loc_dict.keys()
df = get_dataframe('halo_all_cam_labeled_dataset')
print(len(df))
groups, ids = get_groups(df, keys, halo=True, halo_mcsv=halo_mcsv) #list of dicts, each dict corresponds to a group ID, sorted by timestamp
print(len(ids))

2023-11-16 14:50:54,948 - APIRequestor - ERROR - Timeout/Connection error, sleeping for 2 second(s)
ERROR:APIRequestor:Timeout/Connection error, sleeping for 2 second(s)
2023-11-16 14:50:57,302 - APIRequestor - ERROR - API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
ERROR:APIRequestor:API error received | error_code : 403, error_message : {'extra': {'role_name': 'token_exchange'}, 'message': 'Role token_exchange is required to make this API call'}
INFO:botocore.tokens:Loading cached SSO token for brt


70
starting loop
7


In [23]:
groups[0]

{'T01': None,
 'T02': None,
 'T03': None,
 'T04': None,
 'T05': None,
 'T06': None,
 'T07': None,
 'T08': None,
 'T09': '64de71bbc1ffeada36c3a2ef',
 'T10': '64de71c21cdb4debd0823ac8',
 'T11': '64de71c5c1ffeada36c3a2fe',
 'T12': '64de71ba9f3b577ba78f9bc2',
 'T13': '64de71c045147d9d6a85635a',
 'T14': '64de71bdb891b1ff255d3cdf',
 'T15': '64de71bfb891b1ff255d3ce7',
 'T16': '64de71c427e79b61245c3feb',
 'length': 10,
 'group_id': 'b1662dfb57b04a15881c9673a880e053',
 'ts_first': numpy.datetime64('2023-08-14T21:31:10.043000'),
 'ts_last': numpy.datetime64('2023-08-14T21:31:10.088000'),
 'transforms': {'T01': None,
  'T02': None,
  'T03': None,
  'T04': None,
  'T05': None,
  'T06': None,
  'T07': None,
  'T08': None,
  'T09': {'T11': {'intrinsics': {'k': [1430.26953125,
      0.0,
      952.4344482421875,
      0.0,
      1430.26953125,
      610.9564819335938,
      0.0,
      0.0,
      1.0],
     'd': [-0.39831337332725525,
      0.19946284592151642,
      0.00022052868735045195,
      -0.0

In [6]:
all_lefts, unrect_all_clouds = visualize_data(groups[0], keys, pp_path, viz = False, halo = True)  #here, we get 6 image pairs and 6 clouds

(393216, 3)
camera locations: T09 T11
(327680, 3)
camera locations: T10 T11
(393216, 3)
camera locations: T10 T12
(393216, 3)
camera locations: T13 T15
(393216, 3)
camera locations: T14 T16
(327680, 3)
camera locations: T14 T15


In [27]:
import open3d as o3d 
import copy

def get_T(pts, group_info, left_cam_loc, right_cam_loc, transforms):
    
    calib_data = transforms[left_cam_loc][right_cam_loc]
    rot = np.array(calib_data["extrinsics"]['r']).reshape(3, 3)
    tran = list(calib_data["extrinsics"]['t'])
    
    T = np.eye(4)
    T[:3, :3] = rot
    T[:-1, -1] = tran
    print(T)
    return T

def transform_cloud_halo(cloud, left_cam_loc, right_cam_loc, transforms, group_info=None, halo = False, reshape = False):
    pts = o3d.geometry.PointCloud()
    cloud_filt = cloud.copy()
    #if(not reshape):
    #    cloud_filt[cloud_filt[:, -1] > 40] = 0
    pts.points = o3d.utility.Vector3dVector(cloud_filt)

    T = get_T(pts, group_info, left_cam_loc, right_cam_loc, transforms) 

    pts_new = copy.deepcopy(pts).transform(np.linalg.inv(T))
    
    return np.array(pts_new.points)
    
   
def transform_all_clouds_halo(all_clouds, group_info=None, halo = False, reshape = False, ext_transforms = None):


    if(ext_transforms is not None):
        transforms = ext_transforms

    transformed_clouds = {}
    for i, key in enumerate(all_clouds):  
        print(key)
        locs = key.split('_')
        left_loc, right_loc = locs[0], locs[1]

        if(all_clouds[key] is not None):
            new_cloud = transform_cloud_halo(all_clouds[key], left_loc, right_loc, transforms, group_info, halo, reshape)
            print(new_cloud)
            #if(not reshape):
            #    new_cloud[new_cloud[:, -1]<0] = 0
            if(reshape):
                new_cloud = new_cloud.reshape((512, 1024, 3))
            transformed_clouds[key] = new_cloud.copy()
    return transformed_clouds

In [28]:
from helpers.transforms_fixed import visualize_fused_cloud
#from helpers.spatiotemp_viz import visualize_data, plot_cloud
transformed_clouds = transform_all_clouds_halo(unrect_all_clouds.copy(), halo = True, ext_transforms = groups[0]['transforms']) #DONE
cloud_obj = visualize_fused_cloud(transformed_clouds, all_lefts, 0, 6) #DONe

T09_T11
[[ 7.07106647e-01  7.07106915e-01 -6.90542870e-08  1.71826959e+00]
 [ 2.41844941e-01 -2.41844941e-01 -9.39692529e-01  2.77837813e-01]
 [-6.64463103e-01  6.64462817e-01 -3.42020395e-01 -1.19315374e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-139.90136462  118.04555737  102.0465517 ]
 [-139.83751991  118.9996305   102.00758098]
 [-139.77420913  119.94572488  101.96893616]
 ...
 [  -5.85805293    6.05435173   -3.60056188]
 [  -5.83938771    6.03167659   -3.58486323]
 [  -5.82253218    6.01009585   -3.56989156]]
T10_T11
[[-0.24774372  0.96806884 -0.03828539 -0.23817132]
 [ 0.34030924  0.04995518 -0.93898568  0.56100017]
 [-0.90709022 -0.24565668 -0.34181886 -1.96691287]
 [ 0.          0.          0.          1.        ]]
[[-148.32832556 -296.9200278   114.31397333]
 [-148.32832556 -296.9200278   114.31397333]
 [-148.32832556 -296.9200278   114.31397333]
 ...
 [  -6.69051208    2.6200231    -4.26764576]
 [  -6.68164272    2.62327578   -4.26011199]
 [  